# DATATHON 2024 - VIVA Aerobus

## Dependencies

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

# Other models
from datetime import datetime

# LOAD DATASETS

In [ ]:
# Load datasets
flights = pd.read_csv("/work/Filghts TEC_Valid.csv", sep=",")

# WARNING: 2,000,000 Rows
sales = pd.read_csv("/work/Sales TEC_Valid.csv", sep=",")

print(f"Flights shape: {flights.shape}, Sales shape: {sales.shape}")

Flights shape: (245753, 11), Sales shape: (2564361, 5)


In [ ]:
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0
245749,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,2024-04-20 10:45:00,2024-04-20 14:00:00,178,NaN,NaN
245750,9c8970e9836d9c5ef9415bfa93c3f408,XA-VAC,AT,BT,Playa,Ciudad Principal,2023-12-22 12:50:00,2023-12-22 16:10:00,180,163.0,95.0
245751,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,2024-10-04 10:00:00,2024-10-04 13:15:00,240,NaN,NaN


In [ ]:
sales.head(10)

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,7396e43fb94caca87add7bb1f1611e7e,Vivabus,Vivabus_Gdl.-Nvo. C. Camionera,4,320.0
5,34ae909bf3699372fcd66f02af0bdd54,Licores,Ron Bacardi,2,240.0
6,91d3a4c318968b36c16e879b61cb7ed0,Licores,Baileys,1,120.0
7,34867912263acbd7413a4818a8c78a8c,Licores,Corajillo,1,155.0
8,6556ab84c2683b6ba1b84c71aa4ccdcb,Transportaciones CUN,Transfer Cun: Zona Hotelera Sencillo,2,580.0
9,0ba8babe0eff4277dc7c64bb4b929a5a,Galletas,Muffin Integral,1,60.0


### Drop Duplicates

In [ ]:
flights = flights.drop_duplicates("Flight_ID")
flights = flights.set_index("Flight_ID")

### Drop 2025

In [ ]:
print(len(flights))
flights['STD'] = flights['STD'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
flights['STA'] = flights['STA'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))


238055


In [ ]:
flights = flights[~((flights['STD'].dt.year == 2025) | (flights['STA'].dt.year == 2025))]
print(len(flights))

236374


### Convert timestamps to int

In [ ]:
flights['STD-int'] = flights['STD'].apply(lambda x: x.timestamp())
flights['STA-int'] = flights['STA'].apply(lambda x: x.timestamp())

### Split 2023 and 2024

In [ ]:
Flights_2024 = flights[flights["STD"].dt.year == 2024]
Flights_2023 = flights[flights["STD"].dt.year == 2023]

### Drop 2023 if passengers==0

In [ ]:
Flights_2023 = Flights_2023[Flights_2023["Passengers"] > 0]

In [ ]:
print(Flights_2023.shape)
Flights_2023.head()

NameError: name 'Flights_2023' is not defined

In [ ]:
print(Flights_2024.shape)
Flights_2024.head()

(116729, 12)


,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD-int,STA-int
Flight_ID,,,,,,,,,,,,
6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,1.708103e+09,1.708106e+09
bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,BA,AB,Playa,Ciudad Principal,2024-01-12 08:20:00,2024-01-12 09:20:00,186,NaN,NaN,1.705048e+09,1.705051e+09
d77570d39116f81dbb619c5225dd0572,NaN,AK,BO,MX Amigos y Familia,Playa,2024-05-18 14:10:00,2024-05-18 16:15:00,180,NaN,NaN,1.716041e+09,1.716049e+09
85d3b182431d323b856e50e249689981,XA-VIY,AT,AI,Playa,Ciudad Principal,2024-03-18 10:30:00,2024-03-18 13:25:00,186,NaN,NaN,1.710758e+09,1.710768e+09
04e83b220f5513bd6cc636784dcddec6,XA-VXC,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2024-04-05 00:50:00,2024-04-05 04:45:00,240,NaN,NaN,1.712278e+09,1.712292e+09


### Verify Null values

In [ ]:
Flights_2023.isna().sum()

Aeronave            492
DepartureStation      1
ArrivalStation        1
Destination_Type      1
Origin_Type           1
STD                   0
STA                   0
Capacity              0
Passengers            0
Bookings              0
STD-int               0
STA-int               0
dtype: int64

In [ ]:
sales.isna().sum()

Flight_ID      0
ProductType    0
ProductName    0
Quantity       0
TotalSales     0
dtype: int64

In [ ]:
## MAYBE WE CAN USE A PREDICTOR TO FILL THE NAN VALUES OF PASSENGERS AND BOOKINGS.

## FOR NOW, WE CAN DROP THEM OR FILL THEM WITH MEAN OR MODE.

# Auxiliary Dictionaries

# Data Cleaning and Transformations

In [ ]:
Flights_df = Flights_2023.copy()
Sales_df = sales.copy()
#print(Flights_df.head(10))
#print(Sales_df.head(10))

## Clean Flight dataset

### Missing Data Flight

Aeronave is replaced with its mode.

In [ ]:
# Missing Data Strategy
Flights_df.dropna(subset=['DepartureStation','ArrivalStation','Origin_Type', 'Destination_Type'], inplace=True)
numeric_cols = Flights_df.select_dtypes(include='number').columns
categorical_cols = Flights_df.select_dtypes(include='object').columns
Flights_df[numeric_cols] = Flights_df[numeric_cols].fillna(Flights_df[numeric_cols].median())
Flights_df[categorical_cols] = Flights_df[categorical_cols].fillna(Flights_df[categorical_cols].mode().iloc[0])
Flights_df.isna().sum()

Aeronave            0
DepartureStation    0
ArrivalStation      0
Destination_Type    0
Origin_Type         0
STD                 0
STA                 0
Capacity            0
Passengers          0
Bookings            0
STD-int             0
STA-int             0
dtype: int64

### Convert Timestamp to Int values

In [ ]:
# Transform into numeric Departure and Arrival
Flights_df['DUR_HR'] = round(((Flights_df['STA-int'] - Flights_df['STD-int'])/3600), 4)
Flights_df.head(5)

,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD-int,STA-int,DUR_HR
Flight_ID,,,,,,,,,,,,,
ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1.697716e+09,1.697726e+09,2.7500
efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,1.688346e+09,1.688360e+09,4.0000
dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,1.687792e+09,1.687795e+09,0.6667
d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1.676018e+09,1.676023e+09,1.1667
3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,1.694107e+09,1.694110e+09,0.9167


In [ ]:
Flights_df = Flights_df[~(Flights_df["DUR_HR"] > 12)]
print(Flights_df.shape)
Flights_df.head()

(119641, 13)


,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD-int,STA-int,DUR_HR
Flight_ID,,,,,,,,,,,,,
ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1.697716e+09,1.697726e+09,2.7500
efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,1.688346e+09,1.688360e+09,4.0000
dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,1.687792e+09,1.687795e+09,0.6667
d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1.676018e+09,1.676023e+09,1.1667
3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,1.694107e+09,1.694110e+09,0.9167


### Encoding for categorical values

In [ ]:
encoder_stations = OrdinalEncoder()
Flights_df[["DepartureStation_Encoded", "ArrivalStation_Encoded"]] = encoder_stations.fit_transform(Flights_df[["DepartureStation", "ArrivalStation"]])

encoder_destinations = OrdinalEncoder()
Flights_df[["Destination_Encoded", "Origin_Encoded"]] = encoder_destinations.fit_transform(Flights_df[["Destination_Type", "Origin_Type"]])

Stations_Maps = {key:i for i, key in enumerate(encoder_stations.categories_[0])}

Destinations_Maps = {key:i for i, key in enumerate(encoder_destinations.categories_[0])}

#print(encoder_stations.categories_)
#print(encoder_destinations.categories_)
print(Stations_Maps)
print(Destinations_Maps)

{'AB': 0, 'AC': 1, 'AD': 2, 'AE': 3, 'AF': 4, 'AI': 5, 'AJ': 6, 'AK': 7, 'AL': 8, 'AM': 9, 'AO': 10, 'AP': 11, 'AQ': 12, 'AR': 13, 'AS': 14, 'AT': 15, 'AU': 16, 'AV': 17, 'AW': 18, 'AX': 19, 'AY': 20, 'AZ': 21, 'BA': 22, 'BB': 23, 'BC': 24, 'BD': 25, 'BE': 26, 'BF': 27, 'BG': 28, 'BH': 29, 'BI': 30, 'BJ': 31, 'BK': 32, 'BL': 33, 'BM': 34, 'BN': 35, 'BO': 36, 'BP': 37, 'BQ': 38, 'BS': 39, 'BT': 40}
{'Ciudad Fronteriza': 0, 'Ciudad Principal': 1, 'Ecoturismo': 2, 'MX Amigos y Familia': 3, 'Playa': 4}


In [ ]:
Flights_df[["DepartureStation_Encoded", "ArrivalStation_Encoded", "Destination_Encoded", "Origin_Encoded"]].head()

,DepartureStation_Encoded,ArrivalStation_Encoded,Destination_Encoded,Origin_Encoded
Flight_ID,,,,
ab954014077430bd842cfa305a55c0f8,15.0,21.0,0.0,1.0
efd86c996035dacdca7a0ccb2560dda1,34.0,17.0,3.0,0.0
dd0fad3248951d2f71d63e6279aeaa4b,18.0,14.0,3.0,1.0
d0987ee648eea254063bfe2b39571b67,22.0,0.0,4.0,1.0
3b5df8805161ea827d2f2e4298c38e06,6.0,13.0,4.0,3.0


### Verify if passengers > capacity

In [ ]:
Flights_df["Passengers"] = Flights_df[["Capacity", "Passengers"]].apply(lambda x: x["Passengers"] if x["Passengers"] <= x["Capacity"] else x["Capacity"], axis=1)
Flights_df

,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD-int,STA-int,DUR_HR,DepartureStation_Encoded,ArrivalStation_Encoded,Destination_Encoded,Origin_Encoded
Flight_ID,,,,,,,,,,,,,,,,,
ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,1.697716e+09,1.697726e+09,2.7500,15.0,21.0,0.0,1.0
efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,186.0,109.0,1.688346e+09,1.688360e+09,4.0000,34.0,17.0,3.0,0.0
dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,1.687792e+09,1.687795e+09,0.6667,18.0,14.0,3.0,1.0
d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,1.676018e+09,1.676023e+09,1.1667,22.0,0.0,4.0,1.0
3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,1.694107e+09,1.694110e+09,0.9167,6.0,13.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9bbd9f3a2bbcaeae864d17e3ee544d2c,XA-VAE,AT,BT,Playa,Ciudad Principal,2023-12-06 12:50:00,2023-12-06 16:10:00,186,186.0,119.0,1.701867e+09,1.701879e+09,3.3333,15.0,40.0,4.0,1.0
d9ef769565064b1ff1ad90d6d18f69cb,XA-VYF,AO,BT,Playa,Ciudad Principal,2023-12-26 06:20:00,2023-12-26 10:10:00,180,166.0,89.0,1.703572e+09,1.703585e+09,3.8333,10.0,40.0,4.0,1.0
598035669a4f0fa2ce1207977932df9e,XA-VAI,AW,BT,Playa,Ciudad Principal,2023-12-28 10:25:00,2023-12-28 13:50:00,180,142.0,54.0,1.703759e+09,1.703771e+09,3.4167,18.0,40.0,4.0,1.0


## Clean Sales Dataset

In [ ]:
Sales_df

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,7396e43fb94caca87add7bb1f1611e7e,Vivabus,Vivabus_Gdl.-Nvo. C. Camionera,4,320.0
...,...,...,...,...,...
2564356,638cf1904f936c50aea4a6f123ee07cf,Licores,Vino Tinto Sangre De Toro,1,155.0
2564357,04c5a19b8382a888d5c4c7b4932e783e,Licores,Vino Tinto Sangre De Toro,1,155.0
2564358,0291f6b201d2b24717a8ed7ca04f2d90,Licores,Vino Tinto Sangre De Toro,1,155.0
2564359,a10e3ac614fd434d6312897e07727c08,Licores,Vino Tinto Sangre De Toro,1,155.0


### Filter for Food Type Products

In [ ]:
Sales_df[Sales_df["ProductType"] == "Specials"]

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
17,1b7cee1f470cfecfcc91e4e5feb259c5,Specials,Combo Cheve + Carne,1,-12.0
20,5092c234202f0d72a23ba7abda60a9b3,Specials,Tequila + Mezclador,2,-76.0
28,fe89c66c1f793dab9b655ec93ea48581,Specials,Tequila + Mezclador,3,-114.0
31,e7a000706c827505c7f15bacd0aad006,Specials,Tequila + Mezclador,2,-76.0
34,4239a9f071fe0bdec6d94bbc6766f295,Specials,Combo Vino,1,-31.0
...,...,...,...,...,...
2560964,a5c7f0d1c94035150c44dbb102f86c06,Specials,Combo Snack Con Frubotana,1,-6.0
2560965,a3618e4ad8ee5ab8f7dcb678d8e69e0c,Specials,Combo Snack Con Frubotana,1,-6.0
2560966,1ab805c1df428b25c3221e7aec0b20b8,Specials,Combo Snack Con Frubotana,1,-6.0
2560967,5cf9354c39aa720f2333ba7ddd417c38,Specials,Combo Snack Con Frubotana,1,-11.0


In [ ]:
Sales_df["ProductType"].unique()

array(['Botanas', 'Licores', 'Vivabus', 'Transportaciones CUN',
       'Galletas', 'Specials', 'Bebidas Calientes', 'COMBOS CREW',
       'Hertz.', 'OFERTAS ', 'Transportaciones MTY',
       'Transportaciones TLC', 'VIVA PLAY', 'VIVA Taxis', 'Antros',
       'VivaTransfer', 'Alimentos Charter', 'Perecederos', 'Refrescos',
       'Sopas', 'Lacteos'], dtype=object)

In [ ]:
#desired_prod_types = ['Botanas', 'Licores', 'Galletas', 'Bebidas Calientes']
desired_prod_types = ['Botanas', 'Licores', 'Galletas', 'Bebidas Calientes','Alimentos Charter', 'Perecederos', 'Refrescos', 'Sopas', 'Lacteos']


Sales_df = Sales_df[Sales_df['ProductType'].isin(desired_prod_types)]
print(len(Sales_df))

2108296


In [ ]:
print(len(Sales_df["ProductName"].unique()))
for i in Sales_df["ProductName"].unique():
    print(i)

100
Carne Seca Habanero
Jw Red Label 
Jack And Coke
Jw Red Label
Ron Bacardi
Baileys
Corajillo
Muffin Integral
Tequila 7 Leguas Reposado
Arandano Mango Mix
Quaker Granola
Tequila 7 Leguas Blanco
Sol Clamato
Quaker Avena Frutos Rojos
Go Nuts
Arandano
Nutty Berry Mix
Frutos Secos Enchilados
Te Relax
Dip De Queso
Te Frutos Rojos
Ultra Seltzer Frambuesa
Corajillo Baileys 
Vino Tinto Cria Cuervos
Te Manzanilla Jengibre
Baileys 
Nueces De Arbol Mix
Carne Seca Original
Luxury Nut Mix
Galleta De Chispas De Chocolate
Tostitos Nachos Con Dip
Protein Adventure
Vino Blanco Cria Cuervos 
Galleta De Chocolate
Topochico Seltzer Fresa-Guayaba
Te Vainilla
Cafe De Olla
Galleta De Arandano Relleno De Q/Crema
Salsa Botanera
Cafe 19 Cafe Clasico
Licor Charter
Quaker Avena Moras
Quaker Natural Balance
Cuerno Individual Charter
Combo Stl
Eco Holder
Cerveza Charter
Maxi Combo
Chokis
Sprite
Cheetos
Arcoiris
Tostitos
Xx Lager
Xx Ultra
Cafe Costa
Nissin Res
Rancheritos
Amstel Ultra
Nissin Fuego
Tecate Light
Dori

### Group by Flight ID and ProductType

In [ ]:
grouped_sales = Sales_df[Sales_df["ProductType"].isin(desired_prod_types)].groupby(["Flight_ID", "ProductType"]).sum().reset_index()

# Pivot the table to have each desired product type as a column
# NOTE: Can we delete TotalSales?
pivot_sales = grouped_sales.pivot(index="Flight_ID", columns="ProductType", values=["Quantity", "TotalSales"]).fillna(0)

# Flatten the multi-index column
pivot_sales.columns = ['_'.join(col).strip() for col in pivot_sales.columns.values]

# Display the filtered DataFrame
pivot_sales.index.name = 'Flight_ID'

pivot_sales.reset_index(inplace=True)

pivot_sales.head()

,Flight_ID,Quantity_Alimentos Charter,Quantity_Bebidas Calientes,Quantity_Botanas,Quantity_Galletas,Quantity_Lacteos,Quantity_Licores,Quantity_Perecederos,Quantity_Refrescos,Quantity_Sopas,TotalSales_Alimentos Charter,TotalSales_Bebidas Calientes,TotalSales_Botanas,TotalSales_Galletas,TotalSales_Lacteos,TotalSales_Licores,TotalSales_Perecederos,TotalSales_Refrescos,TotalSales_Sopas
0,00004a718edba9d9ef878d08f02ae057,0.0,0.0,13.0,2.0,0.0,1.0,4.0,16.0,3.0,0.0,0.0,645.0,90.0,0.0,72.0,460.0,750.0,135.0
1,0000cd79c0c3a9c309df6064dcacaeea,0.0,3.0,13.0,1.0,1.0,2.0,1.0,17.0,4.0,0.0,135.0,637.0,60.0,42.0,144.0,115.0,798.0,180.0
2,000163f0df9cbfc35c4c06645ec512f6,0.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,90.0,49.0,0.0,0.0,0.0,230.0,48.0,0.0
3,00017be73003a570dd426b155762769c,0.0,6.0,39.0,5.0,2.0,17.0,11.0,37.0,12.0,0.0,270.0,1911.0,222.0,84.0,1505.0,1265.0,1764.0,540.0
4,0001a43836c338f8d8650aefb11672c9,0.0,2.0,29.0,5.0,0.0,19.0,10.0,34.0,13.0,0.0,90.0,1503.0,237.0,0.0,2443.0,1150.0,1632.0,585.0


### Joining datasets

In [ ]:
# Select adequate columns for the flights dataset (Just the relevant (numeric columns))

# Select adequate columns for the sales dataset (Just the relevant (numeric columns))

# Join dataset
df_test = pd.merge(Flights_df, pivot_sales, on='Flight_ID', how='left')
df_test.fillna(0, inplace=True)
df_test.head(5)

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,...,Quantity_Sopas,TotalSales_Alimentos Charter,TotalSales_Bebidas Calientes,TotalSales_Botanas,TotalSales_Galletas,TotalSales_Lacteos,TotalSales_Licores,TotalSales_Perecederos,TotalSales_Refrescos,TotalSales_Sopas
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,...,0.0,0.0,45.0,541.0,45.0,0.0,659.0,460.0,816.0,0.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,186.0,...,2.0,0.0,40.0,143.0,0.0,0.0,350.0,110.0,270.0,90.0
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,...,0.0,0.0,120.0,188.0,168.0,42.0,0.0,330.0,306.0,0.0
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,360.0,0.0,0.0,0.0


In [ ]:
df_test.shape

(119641, 36)

In [ ]:
df_test.isna().sum()

Flight_ID                       0
Aeronave                        0
DepartureStation                0
ArrivalStation                  0
Destination_Type                0
Origin_Type                     0
STD                             0
STA                             0
Capacity                        0
Passengers                      0
Bookings                        0
STD-int                         0
STA-int                         0
DUR_HR                          0
DepartureStation_Encoded        0
ArrivalStation_Encoded          0
Destination_Encoded             0
Origin_Encoded                  0
Quantity_Alimentos Charter      0
Quantity_Bebidas Calientes      0
Quantity_Botanas                0
Quantity_Galletas               0
Quantity_Lacteos                0
Quantity_Licores                0
Quantity_Perecederos            0
Quantity_Refrescos              0
Quantity_Sopas                  0
TotalSales_Alimentos Charter    0
TotalSales_Bebidas Calientes    0
TotalSales_Bot

In [ ]:
df_test.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,...,Quantity_Sopas,TotalSales_Alimentos Charter,TotalSales_Bebidas Calientes,TotalSales_Botanas,TotalSales_Galletas,TotalSales_Lacteos,TotalSales_Licores,TotalSales_Perecederos,TotalSales_Refrescos,TotalSales_Sopas
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,...,0.0,0.0,45.0,541.0,45.0,0.0,659.0,460.0,816.0,0.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,186.0,...,2.0,0.0,40.0,143.0,0.0,0.0,350.0,110.0,270.0,90.0
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,...,0.0,0.0,120.0,188.0,168.0,42.0,0.0,330.0,306.0,0.0
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,360.0,0.0,0.0,0.0


In [ ]:
df_test.to_csv("Dataset_Post_Cleaning.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25b1adb0-2b47-474f-a7b7-c06d723a95f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>